In [94]:
!rm question.sqlite

import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base 
from sqlalchemy import Column, Integer, String, Float, Text, Boolean
from sqlalchemy.orm import Session


import pymysql
pymysql.install_as_MySQLdb()

In [95]:
file = './data/circles.csv'
circle_df = pd.read_csv(file)

In [96]:
circle_df.columns.tolist()

['id',
 'category',
 'test_date',
 'statement',
 'choice_a',
 'choice_b',
 'choice_c',
 'choice_d']

In [97]:
circle_df.head()

,id,category,test_date,statement,choice_a,choice_b,choice_c,choice_d
0,1,Circles,April_2015_1,"The circle in the standard (x,y) coordinate pl...","Lorem ipsum dolor sit amet, ei eum inani adole...","At meis brute nam, odio quas natum eu vel. Med...","Summo adolescens eum et, vel cu homero labore....","Diceret debitis suavitate id pro, nostrud glor..."
1,2,Circles,April_2015_2,"The circle in the standard (x,y) coordinate pl...","Lorem ipsum dolor sit amet, ei eum inani adole...","At meis brute nam, odio quas natum eu vel. Med...","Summo adolescens eum et, vel cu homero labore....","Diceret debitis suavitate id pro, nostrud glor..."
2,3,Circles,April_2015_3,"Points O(0,0) and B(0,3) below lie in the stan...","Lorem ipsum dolor sit amet, ei eum inani adole...","At meis brute nam, odio quas natum eu vel. Med...","Summo adolescens eum et, vel cu homero labore....","Diceret debitis suavitate id pro, nostrud glor..."
3,4,Circles,June_2015_4,The circle shown below has below has diameter ...,"Lorem ipsum dolor sit amet, ei eum inani adole...","At meis brute nam, odio quas natum eu vel. Med...","Summo adolescens eum et, vel cu homero labore....","Diceret debitis suavitate id pro, nostrud glor..."
4,5,Circles,June_2015_5,"Graphed in the same standard (x,y) coordinate ...","Lorem ipsum dolor sit amet, ei eum inani adole...","At meis brute nam, odio quas natum eu vel. Med...","Summo adolescens eum et, vel cu homero labore....","Diceret debitis suavitate id pro, nostrud glor..."


In [98]:
engine = create_engine("sqlite:///question.sqlite")
conn = engine.connect()
Base = declarative_base()

In [99]:
class Questions(Base):
    __tablename__ = "circle"
    id = Column(Integer, primary_key = True)
    category = Column(String(255))
    test_date = Column(String(255))
    statement = Column(String(255))
    choice_a = Column(String(255))
    choice_b = Column(String(255))
    choice_c = Column(String(255))
    choice_d = Column(String(255))

In [100]:
circle_dict = circle_df.to_dict(orient='records')

In [101]:
Base.metadata.create_all(engine)
metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)

table = sqlalchemy.Table('circle', metadata, autoload=True)
conn.execute(table.delete())
conn.execute(table.insert(), circle_dict)


/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  


In [102]:
conn.execute("SELECT * FROM circle LIMIT 2").fetchall()

[(1, 'Circles', 'April_2015_1', 'The circle in the standard (x,y) coordinate plane below has center (-8.5, 7.5) and has radius 5 coordinate units.', 'Lorem ipsum dolor sit amet, ei eum inani adolescens temporibus, omnesque voluptatibus mediocritatem nec te. ', 'At meis brute nam, odio quas natum eu vel. Mediocrem partiendo id cum, epicuri blandit ut eum. ', 'Summo adolescens eum et, vel cu homero labore. Pri reprehendunt signiferumque an. In sea case perpetua.', 'Diceret debitis suavitate id pro, nostrud gloriatur mel at. '),
 (2, 'Circles', 'April_2015_2', 'The circle in the standard (x,y) coordinate plane below has center (-8.5, 7.5) and has radius 5 coordinate units.', 'Lorem ipsum dolor sit amet, ei eum inani adolescens temporibus, omnesque voluptatibus mediocritatem nec te. ', 'At meis brute nam, odio quas natum eu vel. Mediocrem partiendo id cum, epicuri blandit ut eum. ', 'Summo adolescens eum et, vel cu homero labore. Pri reprehendunt signiferumque an. In sea case perpetua.', '

In [112]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, Query
import json

engine = create_engine('sqlite:///data/question.sqlite', convert_unicode=True, echo=False)
Base = declarative_base()
Base.metadata.reflect(engine)


class Questions(Base):
    __table__ = Base.metadata.tables['circle']


session = scoped_session(sessionmaker(bind=engine))


def questions(input_data):

    questionReturn = session.query(Questions.statement,
                                   Questions.test_date,
                                   Questions.choice_a,
                                   Questions.choice_b,
                                   Questions.choice_c,
                                   Questions.choice_d).filter(Questions.test_date == input_data)
    
    dataDict = [{'Question': each[0],
                 'Test_Date': each[1],
                 'Answer_1': each[2],
                 'Answer_2': each[3],
                 'Answer_3': each[4],
                 'Answer_5': each[5]
                } for each in questionReturn]

    return dataDict

In [113]:
questions('April_2015_1')[0]['Question']

'The circle in the standard (x,y) coordinate plane below has center (-8.5, 7.5) and has radius 5 coordinate units.'